# Debate a base
#### Update script (Parlamint 3.0)

In [6]:
translated_set = r"D:\Debate_a_Base\Data\Parlamint_3.0\Overkoepelende_taal\translated_extract"
original_set = r"D:\Debate_a_Base\Data\Parlamint_3.0\Originele_taal\original_extract"
csv_set = r"D:\Debate_a_Base\Data\Parlamint_3.0\Csv_set"

### Helpers

In [1]:
# Imports
import os

import warnings
warnings.filterwarnings('ignore')

# import pandas as pd
import re
import csv
# from unidecode import unidecode
from bs4 import BeautifulSoup as bs
from joblib import Parallel, delayed

# from dataclasses import dataclass

# import tarfile
# import shutil

# import time

In [4]:
selected_countries = ['ParlaMint-AT-en.TEI.ana',
                      'ParlaMint-BA-en.TEI.ana',
                      'ParlaMint-BE-en.TEI.ana',
                      'ParlaMint-BG-en.TEI.ana',
                      'ParlaMint-CZ-en.TEI.ana',
                      'ParlaMint-DK-en.TEI.ana',
                      'ParlaMint-EE-en.TEI.ana',
                      'ParlaMint-ES-CT-en.TEI.ana',
                      'ParlaMint-ES-GA-en.TEI.ana',
                      'ParlaMint-FR-en.TEI.ana',
                      'ParlaMint-GB-en.TEI.ana',
                      'ParlaMint-GR-en.TEI.ana',
                      'ParlaMint-HR-en.TEI.ana',
                      'ParlaMint-HU-en.TEI.ana',
                      'ParlaMint-IS-en.TEI.ana',
                      'ParlaMint-IT-en.TEI.ana',
                      'ParlaMint-LV-en.TEI.ana',
                      'ParlaMint-NL-en.TEI.ana',
                      'ParlaMint-NO-en.TEI.ana',
                      'ParlaMint-PL-en.TEI.ana',
                      'ParlaMint-PT-en.TEI.ana',
                      'ParlaMint-RS-en.TEI.ana',
                      'ParlaMint-SE-en.TEI.ana',
                      'ParlaMint-SI-en.TEI.ana',
                      'ParlaMint-TR-en.TEI.ana',
                      'ParlaMint-UA-en.TEI.ana']

In [5]:
# Retrieve the file structure
def get_xml_files(country_selection, target=translated_set):
    os.chdir(target)
    
    country_return_list = []
    
    # loop door alle folders die hierboven zijn geprint
    for country in os.listdir():

        # filter op specifiek land (IN BOX 2)
        if country in country_selection:
            paths_dict = {}

            # ga door alle inhoud van de landfolder heen
            for root, dirs, files in os.walk(country):
                file_data = []
                
                # loop door files van een folder
                for file in files:

                    #filter alleen de xml files
                    if ".xml" in file and not "~" in file:
                        file_data.append(file)

                # filter onzin uit de dict voor makkelijkere processing later
                if not "Schema" in root:
                    paths_dict[root] = file_data

            country_return_list.append(paths_dict)
        
    return country_return_list

In [ ]:
# functie testruimte
# selected_countries = ["ParlaMint-NL-en.TEI.ana"]

# answer = get_xml_files(selected_countries)

# print(answer)

In [ ]:
# vraag een dict met k (userline) v (sentence)
def get_translated_dict(path):
    with open(path, "r", encoding="utf-8") as file:
        # read each line in the file, readlines() returns a list of lines
        content = file.readlines()
        # combine the lines in the list into a string
        content = "".join(content)
        bs_content = bs(content, "lxml")

    # ga door alle segmenten info heen
    userlines = bs_content.find_all("u")
    segments = dict()
    for line in userlines:
        seg = line.get("xml:id")
        
        # combineer de woorden tot een zin
        combined_line = ""
        for part_of_line in line.find_all("seg"):
            combined_line += part_of_line.text
            combined_line = combined_line.replace("\n\n", " ").replace("\n", " ").replace("  ", " ")
        
        # zet de interpunctie goed
        start_index = 0
        punctuated_line = ""
        for dot in re.finditer("\.|,|\?|\!|\)|;", combined_line):
            index = dot.start()

            if combined_line[index - 1] == " ":
                punctuated_line += combined_line[start_index:(index - 1)] + combined_line[index]

                start_index = (index + 1)
        
        # voorkom dat zinnen zonder interpunctie missing gaan
        if punctuated_line == "":
            punctuated_line = combined_line

        # haal de spatie op het begin weg
        if len(punctuated_line) > 0 and punctuated_line[0] == " ":
            punctuated_line = punctuated_line[1:]
            
        # sla de zin op met seg als key
        segments[seg] = punctuated_line.replace("  ", " ")

    return segments

In [ ]:
# functie testruimte
# translate_path_test = r"C:\Users\Asher\Documents\School\_Scriptie\Data\Parlamint_3.0\Overkoepelende_taal\translated_extract\ParlaMint-NL-en.TEI.ana\2014\ParlaMint-NL-en_2014-04-16-tweedekamer-2.ana.xml"
# translate_path_test = r"C:\Users\Asher\Documents\School\_Scriptie\Data\Parlamint_3.0\Overkoepelende_taal\translated_extract\ParlaMint-BA-en.TEI.ana\2009\ParlaMint-BA-en_2009-09-16-0.ana.xml"
# translate_path_test = r"D:\Debate_a_Base\Data\Parlamint_3.0\Overkoepelende_taal\translated_extract\ParlaMint-BG-en.TEI.ana\2014\ParlaMint-BG-en_2014-11-05.ana.xml"

# for k, v in get_translated_dict(translate_path_test).items():
#     print(k, "\n<" + v + ">\n\n")

### Step 1: Data to .csv files
- The following code puts all country but GB into .csv files
- Run the "Scrap great britain 3.0.csv" file to put GB into .csv file format

In [ ]:
# maak een folder per land aan (als dat nog niet gedaan is)
def verify_country_folders():
    for country in os.listdir(translated_set):
        
        country_short = country.replace("ParlaMint-", "").replace("-en.TEI.ana", "")
        if country_short not in os.listdir(csv_set):
            os.mkdir(os.path.join(csv_set, country_short))
            
            print(f"[Info]: {country_short} folder created")
    
    return f"[Info]: .csv folders exist"
        
verify_country_folders()

In [ ]:
# maak een csv waar segments zijn gemerged per file
def create_userline_csv(file, content):
    os.chdir(csv_set)
    
    with open(file, 'w', newline='', encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["key", "value"])
        
        for k, v in content.items():
            
            w.writerow([k, v])

In [ ]:
# functie testruimte
# userline_test_file = "NL\\text_xd.csv"

# translate_path_test = r"D:\Debate_a_Base\Data\Parlamint_3.0\Overkoepelende_taal\translated_extract\ParlaMint-BG-en.TEI.ana\2014\ParlaMint-BG-en_2014-11-05.ana.xml"

# create_userline_csv(userline_test_file, get_translated_dict(translate_path_test))

In [ ]:
# crates .csv files from all translated content of the given countries
def create_csvs(selected_countries, cnt=0):
    
    # vraag alle xml files op van de gegeven land(en)
    countries = get_xml_files(selected_countries)
    
    # loop door elk land heen
    for country in countries:
        
        # split hoofdfolder van mogelijke subfolders van jaren
        rootfolder = (k := next(iter(country)), country.pop(k))
        
        # country label (bv: GB of NL of BE)
        cty = rootfolder[0].replace("ParlaMint-", "").replace("-en.TEI.ana", "")
        
        def multithread_year_folder(year_folder):
            for file in country[year_folder]:
                path = os.path.join(translated_set, year_folder, file)

                create_userline_csv(f'{cty}\\{file.replace(".ana.xml", ".csv")}', get_translated_dict(path))

            print(f'[Created]: {year_folder} .csv files from {cty}')

        # process every yearfolder at the same time per country
        Parallel(n_jobs=len(country))(delayed(multithread_year_folder)(year_folder) for year_folder in country)
        
        print(f'[Created]: {cty} .csv files')
    
    return

In [ ]:
# creeer alle .csv's
# NOTE: dit is op GB na, die kan gedaan worden met "Scrap great britain 3.0"
# create_csvs(selected_countries)

### Step 2: Get the word counts for each date
- Run the "Csv word counter 3.0" file

### Step 3: Start creating Ngrams
- Run the "Ngrams indexing 3.0" file

### Multiprocessing
- Just some practice

In [ ]:
# https://measurespace.medium.com/use-joblib-to-run-your-python-code-in-parallel-ad82abb26954
# import time, math
# def my_fun(i):
#     """ We define a simple function here.
#     """
#     r = i.split("-")[1]
#     return r

In [ ]:
# start = time.time()
# # n_jobs is the number of parallel jobs
# x = Parallel(n_jobs=5)(delayed(my_fun)(i) for i in selected_countries)
# end = time.time()
# print('{:.4f} s'.format(end-start))
# print(x)